<a href="https://colab.research.google.com/github/Uthmanic/HAMOYE-DataScience-Internship/blob/master/Hamoye_Stage_D(Planet_Understanding_the_Amazon_from_SpaceAmazon).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install kaggle
!pip install kaggle

In [2]:
!mkdir .kaggle

In [3]:
import json
token = {"username":"tijaniuthman","key":"42a091ba934aa0b6d9ef884eb7f52567"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [14]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [15]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [16]:
!chmod 600 /root/.kaggle/kaggle.json

In [17]:
!kaggle datasets list

ref                                                    title                                              size  lastUpdated          downloadCount  
-----------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
anmolkumar/health-insurance-cross-sell-prediction      Health Insurance Cross Sell Prediction 🏠 🏥          6MB  2020-09-11 18:39:31           1213  
shivan118/healthcare-analytics                         Healthcare Analytics                                2MB  2020-09-13 17:40:05            658  
Cornell-University/arxiv                               arXiv Dataset                                     885MB  2020-09-14 16:54:47           2872  
nipunarora8/age-gender-and-ethnicity-face-data-csv     AGE, GENDER AND ETHNICITY (FACE DATA) CSV          63MB  2020-09-02 13:46:38            384  
datatattle/covid-19-nlp-text-classification            Coronavirus tweets NLP - Text Classification       

In [18]:
#search for the dataset 
!kaggle datasets list -s planets-dataset

ref                                               title                                     size  lastUpdated          downloadCount  
------------------------------------------------  ---------------------------------------  -----  -------------------  -------------  
nikitarom/planets-dataset                         planets_dataset                            1GB  2020-02-09 22:01:31           1939  
mrisdal/open-exoplanet-catalogue                  Open Exoplanet Catalogue                 123KB  2017-06-08 19:59:22           2805  
eduardowoj/exoplanets-database                    Exoplanets Database                      281KB  2018-02-28 20:56:05            852  
muhakabartay/markmarkohkeplerconfirmedplanets     Kepler Confirmed Planets                 521KB  2019-12-26 20:08:20             17  
nasa/kepler-exoplanet-search-results              Kepler Exoplanet Search Results            1MB  2017-10-10 18:26:59           4007  
jsphyg/star-wars                                  Star 

In [19]:
#download dataset
!kaggle datasets download -d nikitarom/planets-dataset -p /content

 99% 1.49G/1.50G [00:41<00:00, 45.2MB/s]
100% 1.50G/1.50G [00:41<00:00, 38.3MB/s]


In [30]:
#unzip the dataset
!unzip \*.zip&> /dev/null

^C


In [1]:
#Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import cv2
from PIL import Image
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import fbeta_score
import time
%matplotlib inline

pal = sns.color_palette()

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#check present working directory
! pwd

/content


In [3]:
#read the train classes csv file and save a copy
df_train_classes= pd.read_csv('/content/planet/planet/train_classes.csv')
df_train = df_train_classes.copy()
df_train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [4]:
#read the sample submission classes csv file and save a copy
df_test_classes = pd.read_csv('/content/planet/planet/sample_submission.csv')
df_test = df_test_classes.copy()
df_test.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [5]:
#total train data points
!ls /content/planet/planet/train-jpg | wc -l

40479


In [6]:
test1 = !ls /content/planet/planet/test-jpg | wc -l
test2 = !ls /content/test-jpg-additional/test-jpg-additional | wc -l
assert df_test.shape[0] == float(test1[0])+float(test2[0])

In [7]:
#list of unique labels/tags 
labels = df_train['tags'].apply(lambda x: x.split(' '))
from collections import Counter, defaultdict
counts = defaultdict(int)
for l in labels:
    for l2 in l:
        counts[l2] += 1
               
tag_list=list(counts.keys())
tag_list 

['haze',
 'primary',
 'agriculture',
 'clear',
 'water',
 'habitation',
 'road',
 'cultivation',
 'slash_burn',
 'cloudy',
 'partly_cloudy',
 'conventional_mine',
 'bare_ground',
 'artisinal_mine',
 'blooming',
 'selective_logging',
 'blow_down']

In [8]:
#total unique labels
len(tag_list)

17

In [9]:
#assign numerical value to each label using dictionary
label_map = {i:j for j, i in enumerate(tag_list)}
label_map

{'agriculture': 2,
 'artisinal_mine': 13,
 'bare_ground': 12,
 'blooming': 14,
 'blow_down': 16,
 'clear': 3,
 'cloudy': 9,
 'conventional_mine': 11,
 'cultivation': 7,
 'habitation': 5,
 'haze': 0,
 'partly_cloudy': 10,
 'primary': 1,
 'road': 6,
 'selective_logging': 15,
 'slash_burn': 8,
 'water': 4}

In [10]:
# One hot encode the training labels. Convert the images into pixels and resize them
X_train, Y_train = [], []
for img, label in tqdm(df_train.values, miniters = 1000):
  target = np.zeros(17)
  for tag in label.split(' '):
    target[label_map[tag]]=1
  X_train.append(cv2.resize(cv2.imread('/content/planet/planet/train-jpg/{}.jpg'.format(img)), (64,64)))
  Y_train.append(target)

100%|██████████| 40479/40479 [00:54<00:00, 739.76it/s]


In [11]:
#convert the test images to pixels and resize them as well
X_test=[]
for img, label in tqdm(df_test[:40669].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('/content/planet/planet/test-jpg/{}.jpg'.format(img)), (64,64)))
for img, label in tqdm(df_test[40669:].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('/content/test-jpg-additional/test-jpg-additional/{}.jpg'.format(img)), (64,64)))
x_test = np.array(X_test, np.float16)/255

100%|██████████| 20522/20522 [00:26<00:00, 786.91it/s]


In [12]:
#Confirm the dimensions
len(X_test), len(X_train), len(Y_train)

(61191, 40479, 40479)

In [13]:
import sys
sys.getsizeof(X_train)

361288

In [14]:
#import and initialise Garbage Collector
import gc
gc.collect()

11

In [15]:
#Change lists to numpy arrays and normalize
x_train = np.array(X_train, np.float16)/255
y_train = np.array(Y_train, np.uint8)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [16]:
#delete X_test, X_train, Y_train to free up space
del(X_test, X_train, Y_train)

In [17]:
#Split the data into 5 folds and train on 4 folds while validating on 1 fold

yfull_test = []
yfull_train = []
X_train_, X_val_, Y_train_, Y_val_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 1)

In [18]:
#import and initialise Garbage Collector
import gc
gc.collect()

0

In [19]:
#Build a CNN model
from keras.applications.vgg19 import VGG19
from keras.optimizers import Adam

input_size = 64
input_channels = 3
batch_size = 64

# Add model parameters including dropout, layers and activation function
base_model = VGG19(include_top=False,
                   weights='imagenet',
                   input_shape=(input_size, input_size, input_channels))

model = Sequential()
model.add(BatchNormalization(input_shape=(input_size, input_size, input_channels)))

model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(17, activation='sigmoid'))

In [20]:
#import and initialise Garbage Collector
import gc
gc.collect()

0

In [21]:
# define model training optimizer parameters
opt = Adam(lr=1e-4)
model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0),
                ModelCheckpoint(filepath='weights/best_weights',
                                 save_best_only=True,
                                 save_weights_only=True)]
model.summary()




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 64, 64, 3)         12        
_________________________________________________________________
vgg19 (Functional)           (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 17)                34833     
Total params: 20,059,229
Trainable params: 20,059,223
Non-trainable params: 6
_________________________________________________________________


In [22]:
gc.collect()

0

In [23]:
#Try a combination of epoch lengths and learning rates

model.fit(x = X_train_, y= Y_train_, validation_data=(X_val_, Y_val_),batch_size=64,verbose=2, epochs=10,callbacks=callbacks,shuffle=True)

p_val = model.predict(X_val_, batch_size = 64, verbose=2)
print(fbeta_score(Y_val_, np.array(p_val) > 0.2, beta=2, average='samples')) #Check the model performance on the validation set

p_train = model.predict(x_train, batch_size =64, verbose=2) #save the training predictions
yfull_train.append(p_train)
        
p_test = model.predict(x_test, batch_size = 64, verbose=2) #save the test predictions
yfull_test.append(p_test)

Epoch 1/15
405/405 - 25s - loss: 0.1580 - accuracy: 0.6921 - val_loss: 0.1232 - val_accuracy: 0.7906
Epoch 2/15
405/405 - 23s - loss: 0.1163 - accuracy: 0.7145 - val_loss: 0.1087 - val_accuracy: 0.8502
Epoch 3/15
405/405 - 23s - loss: 0.1075 - accuracy: 0.7342 - val_loss: 0.1054 - val_accuracy: 0.7815
Epoch 4/15
405/405 - 23s - loss: 0.1008 - accuracy: 0.7394 - val_loss: 0.1028 - val_accuracy: 0.7273
Epoch 5/15
405/405 - 23s - loss: 0.0954 - accuracy: 0.7207 - val_loss: 0.1024 - val_accuracy: 0.8376
Epoch 6/15
405/405 - 23s - loss: 0.0902 - accuracy: 0.7314 - val_loss: 0.1139 - val_accuracy: 0.8067
Epoch 7/15
405/405 - 23s - loss: 0.0828 - accuracy: 0.7064 - val_loss: 0.1034 - val_accuracy: 0.6960
203/203 - 2s
0.9119118308929165
506/506 - 8s
957/957 - 15s


In [24]:
#import and initialise Garbage Collector
import gc
gc.collect()

2967

In [25]:
#make predictions
predictions = model.predict(x_test, batch_size = 32)
predictions

array([[2.03687279e-03, 9.99922156e-01, 2.83319782e-03, ...,
        6.13635313e-03, 5.29130339e-04, 3.14486533e-04],
       [7.15181290e-04, 9.99950171e-01, 5.41629829e-03, ...,
        9.14758220e-02, 7.18053477e-03, 2.10611592e-03],
       [1.79610646e-03, 9.94473040e-01, 2.55280230e-02, ...,
        5.46605908e-04, 1.81650030e-04, 3.53933719e-04],
       ...,
       [3.55643366e-04, 9.99871254e-01, 9.68208257e-03, ...,
        2.01380439e-03, 5.03386196e-04, 1.78211980e-04],
       [7.99031928e-03, 1.44288745e-02, 1.02722552e-03, ...,
        1.29998341e-04, 1.68812003e-05, 3.27370581e-05],
       [7.49441516e-03, 7.66544223e-01, 2.58286536e-01, ...,
        1.97319823e-05, 1.09845336e-04, 3.56147539e-05]], dtype=float32)

In [26]:
#predictions dataframe
pred = pd.DataFrame(predictions, columns =  tag_list)
pred

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,0.002037,0.999922,0.002833,0.996940,0.000862,0.000867,0.000563,0.000343,0.000008,0.000037,0.000536,0.000003,0.000091,0.000003,0.006136,0.000529,0.000314
1,0.000715,0.999950,0.005416,0.997844,0.001017,0.001851,0.001209,0.000783,0.000036,0.000038,0.000830,0.000007,0.000230,0.000009,0.091476,0.007181,0.002106
2,0.001796,0.994473,0.025528,0.001813,0.063637,0.002667,0.021063,0.001763,0.000062,0.003707,0.994457,0.000093,0.000793,0.000113,0.000547,0.000182,0.000354
3,0.006248,0.999798,0.025696,0.949387,0.002227,0.001672,0.001170,0.004785,0.000084,0.000144,0.030627,0.000024,0.000532,0.000019,0.005454,0.001036,0.001387
4,0.007452,0.544485,0.034021,0.003442,0.054959,0.007817,0.022010,0.005230,0.000518,0.486708,0.662707,0.000793,0.003699,0.001350,0.001997,0.000929,0.000771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.005558,0.042965,0.002798,0.001489,0.004891,0.000635,0.001748,0.000575,0.000027,0.965379,0.031159,0.000026,0.000325,0.000085,0.000297,0.000054,0.000086
61187,0.000427,0.999723,0.270662,0.997778,0.999289,0.253440,0.010369,0.345925,0.001636,0.000021,0.001029,0.000048,0.010031,0.000960,0.000353,0.000869,0.000037
61188,0.000356,0.999871,0.009682,0.999623,0.047336,0.001771,0.002455,0.000567,0.000013,0.000018,0.000042,0.000005,0.000212,0.000006,0.002014,0.000503,0.000178
61189,0.007990,0.014429,0.001027,0.021951,0.011383,0.000404,0.000709,0.000157,0.000010,0.971190,0.000365,0.000005,0.000114,0.000017,0.000130,0.000017,0.000033


In [27]:
#Write a loop to assign each value to its predicted label
preds = []
for i in tqdm(range(pred.shape[0]), miniters=1000):
    a = pred.iloc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

100%|██████████| 61191/61191 [02:43<00:00, 373.16it/s]


In [28]:
#Assign the predictions to the tags column of the df_test data frame
df_test['tags'] = preds 
df_test.to_csv('uthman_9th_Submission.csv', index = False) #Export to csv
df_test.head()

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary clear
4,test_4,primary cloudy partly_cloudy


In [29]:
#download submission file
from google.colab import files
files.download('uthman_9th_Submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>